In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from config import PGHOST, PGDATABASE, PGUSER, PGPASSWORD

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM

In [2]:
# Define a function to connect to AWS database instance

def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+PGHOST +" port="+ "5432" +" dbname="+PGDATABASE +" user=" + PGUSER \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [3]:
conn, cursor = connect()

Connected!


In [15]:
# Get the data, read SQL query into data frame
bb_data = pd.read_sql_query("SELECT * FROM blackberry", con=conn)
bb_df = pd.DataFrame(bb_data)
bb_df.head()

,Date,open_price,high_price,low_price,close_price,adj_close,volume
0,1999-02-04,2.145833,2.166667,1.895833,1.924479,1.924479,16788600
1,1999-02-05,1.929688,1.947917,1.822917,1.833333,1.833333,3053400
2,1999-02-08,1.854167,1.927083,1.783854,1.812500,1.812500,1548000
3,1999-02-09,1.822917,1.833333,1.656250,1.666667,1.666667,3501600
4,1999-02-10,1.708333,1.708333,1.604167,1.677083,1.677083,1597200


In [16]:
ma_day = [10, 20, 50]

for ma in ma_day:
    #for company in companies:
    column_name = f"MA for {ma} days"
    bb_df[column_name] = bb_df['close_price'].rolling(ma).mean()


bb_df

,Date,open_price,high_price,low_price,close_price,adj_close,volume,MA for 10 days,MA for 20 days,MA for 50 days
0,1999-02-04,2.145833,2.166667,1.895833,1.924479,1.924479,16788600,NaN,NaN,NaN
1,1999-02-05,1.929688,1.947917,1.822917,1.833333,1.833333,3053400,NaN,NaN,NaN
2,1999-02-08,1.854167,1.927083,1.783854,1.812500,1.812500,1548000,NaN,NaN,NaN
3,1999-02-09,1.822917,1.833333,1.656250,1.666667,1.666667,3501600,NaN,NaN,NaN
4,1999-02-10,1.708333,1.708333,1.604167,1.677083,1.677083,1597200,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5940,2022-09-13,5.900000,5.990000,5.830000,5.930000,5.930000,6715400,5.869,6.1370,6.1540
5941,2022-09-14,5.940000,6.050000,5.830000,5.960000,5.960000,7881800,5.861,6.0840,6.1604
5942,2022-09-15,5.900000,6.100000,5.890000,6.050000,6.050000,7950900,5.872,6.0480,6.1646
5943,2022-09-16,5.920000,5.920000,5.660000,5.720000,5.720000,5643200,5.868,5.9965,6.1578


In [19]:
bb_df = bb_df.loc["2000-01-01":]
bb_df

,Date,open_price,high_price,low_price,close_price,adj_close,volume,MA for 10 days,MA for 20 days,MA for 50 days
2020,2007-02-16,45.146667,46.350000,44.806667,46.273335,46.273335,18357600,45.437667,43.788500,43.796933
2021,2007-02-20,45.886665,47.456670,45.513332,47.206670,47.206670,25712700,45.598000,44.128834,43.834733
2022,2007-02-21,46.916668,47.623333,46.473330,47.326668,47.326668,24587700,45.744000,44.382667,43.878333
2023,2007-02-22,47.483334,47.493332,46.273335,46.766666,46.766666,20338200,45.864334,44.592167,43.955333
2024,2007-02-23,46.610000,46.950000,46.190000,46.666668,46.666668,14927100,45.917668,44.808167,44.037400
...,...,...,...,...,...,...,...,...,...,...
5940,2022-09-13,5.900000,5.990000,5.830000,5.930000,5.930000,6715400,5.869000,6.137000,6.154000
5941,2022-09-14,5.940000,6.050000,5.830000,5.960000,5.960000,7881800,5.861000,6.084000,6.160400
5942,2022-09-15,5.900000,6.100000,5.890000,6.050000,6.050000,7950900,5.872000,6.048000,6.164600
5943,2022-09-16,5.920000,5.920000,5.660000,5.720000,5.720000,5643200,5.868000,5.996500,6.157800


In [7]:
# Get the data, read SQL query into data frame
# gg_data = pd.read_sql_query("SELECT * FROM google", con=conn)
# gg_df = pd.DataFrame(gg_data)
# gg_df

In [8]:
# Get the data, read SQL query into data frame
# nf_data = pd.read_sql_query("SELECT * FROM netflix", con=conn)
# nf_df = pd.DataFrame(nf_data)
# nf_df.head()

In [9]:
# Check the rows and columns of dfs
# print(bb_df.shape)
# print(gg_df.shape)
# print(nf_df.shape)

In [10]:
# Check if there is null values
bb_df.isnull().sum()

0

In [11]:
# Check the data types in df
bb_df.info()

AttributeError: 'Series' object has no attribute 'info'

In [ ]:
# gg_df.isnull().sum()

In [ ]:
# nf_df.isnull().sum()

In [ ]:
# Change [Date] data type to data time

bb_df['Date'] = pd.to_datetime(bb_df.Date)
# gg_df['Date'] = pd.to_datetime(gg_df.Date)
# nf_df['Date'] = pd.to_datetime(nf_df.Date)

In [ ]:
bb_df.info()

In [ ]:
# Delete 'Adj Close' column and check df

# bb_df = bb_df.drop('adj_close', axis=1)
# gg_df = gg_df.drop('adj_close', axis=1)
# nf_df = nf_df.drop('adj_close', axis=1)

In [ ]:
bb_df.set_index('Date')
# gg_df.set_index('Date')
# nf_df.set_index('Date')

In [ ]:
# Vistulize the stock price trend by using 'Close' column
plt.plot(bb_df['Date'], bb_df['close_price'],label='Close')
# plt.plot(bb_df['MA for 10 days'], color = "green")
# plt.plot(bb_df['MA for 20 days'], color = "orange")
# plt.plot(bb_df['MA for 50 days'], color = "purple")
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('BB History Stock Price')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Vistulize the stock price trend by using 'Close' column
plt.plot(gg_df['Date'], gg_df['close_price'], label='Close')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Google History Stock Price')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Vistulize the stock price trend by using 'Close' column
plt.plot(nf_df['Date'], nf_df['close_price'], label='Close')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Netflix History Stock Price')
plt.grid()
plt.legend()
plt.show()

# Multiple Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
# Split our preprocessed data into our features and target arrays

X = bb_df[['open_price','high_price','low_price','volume']].values
y = bb_df['close_price'].values

In [ ]:
# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Check the shape of X_train and X_test

print(X_train.shape)
print(X_test.shape)

In [ ]:
# Build linear reg model

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

In [ ]:
# Check the model score

lr_model.score(X_train, y_train)

In [ ]:
# Make stock price prediction

prediction = lr_model.predict(X_test)

print(prediction.shape)
print(prediction)

In [ ]:
# Compare y_test and our prediction value

pred_df = pd.DataFrame({'Actual Price':y_test, 'Predicted Price':prediction})

pred_df.head(20)

In [ ]:
# Plot the actual and predicted price
# from the above data frame, we can see the difference between 'actual price' and 'predicted price' is tiny
# if using line graph, the two lines will be overlapped to each other.
# so we choose to use bar graph to show the difference of first 20 rows of data frame

fig_df = pred_df.loc[pred_df.index<20]

x = np.arange(len(fig_df['Actual Price']))  # the label locations
width = 0.35  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, fig_df['Actual Price'], width, label='Actual')
rects2 = ax.bar(x + width/2, fig_df['Predicted Price'], width, label='Predicted')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Time')
ax.set_ylabel('Stock Price')
ax.set_title('Stock Price predicted by Linear Regression Model')
ax.legend()
fig.tight_layout()
plt.show()

In [ ]:
print(lr_model.coef_)
print(lr_model.intercept_)

In [ ]:
# Calculate the MSE for our model
# The mean squared error (MSE) tells you how close a regression line is to a set of points
# return a non-negative floating point value (the best value is 0.0)

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, prediction)

# Using more popular LSTM model to predict BB stock price

In [ ]:
# Choose "Closed" price column as our input variable
close_df= bb_df['close_price']

data_len = round(len(close_df) * 0.8)

close_df_train = close_df.loc[close_df.index < data_len]
close_df_test = close_df.loc[close_df.index >= data_len]

#print(close_df_train.head())
#print()
#print(close_df_test.head())
close_df

In [ ]:
# Scale the training data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(close_df_train.values.reshape(-1,1))
scaled_data.shape

In [ ]:
# Use the previous 30 days 'close' price to predict the 31st day's price
previous_days = 30

In [ ]:
# Splite data into X_train and y_train
X_train = []
y_train = []

for i in range(previous_days, len(scaled_data)):
    X_train.append(scaled_data[i-previous_days:i, 0])
    y_train.append(scaled_data[i, 0])

In [ ]:
# Convert the X_train and y_train to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train.shape)
print(y_train)

In [ ]:
# Reshape the X_train since LSTM needs 3-dimensional inputs
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

In [ ]:
# Build the LSTM model
model = Sequential()
# Add LSTM layer
model.add(LSTM(units = 50, return_sequences=True, input_shape = (X_train.shape[1],1)))
model.add(LSTM(units = 50, return_sequences=True))
model.add(LSTM(units = 50))
# prediction output layer
model.add(Dense(units=1))

In [ ]:
# compile and train the data
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size =32, epochs=20)

In [ ]:
model.summary()

In [ ]:
#### Test model ####

# Scale the testing data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_test_data = scaler.fit_transform(close_df_test.values.reshape(-1,1))
scaled_test_data.shape

In [ ]:
# Create X_test and y_test
X_test = []
y_test = []

for i in range(previous_days, len(scaled_test_data)):
    X_test.append(scaled_test_data[i-previous_days:i, 0])
    y_test.append(scaled_test_data[i, 0])

In [ ]:
# Convert the X_test and y_test to numpy arrays
X_test = np.array(X_test)
print(X_test.shape)

In [ ]:
# reshape
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print(X_test.shape)
print(y_test)

In [ ]:
# Predict the data

prediction = model.predict(X_test)
prediction = scaler.inverse_transform(prediction)

In [ ]:
# plot test predictions

plt.figure(figsize=(12, 8))
plt.plot(y_test, color='red', label='Actual')
plt.plot(prediction, color='green', label='Predicted')
plt.xlabel('Date')
plt.ylabel('Stock Price')

plt.title("Stock Price predicted by LSTM Model")
plt.grid()
plt.legend()
plt.show()

# Using LSTM model to predict GOOGLE stock price

In [ ]:
# Choose "Closed" price column as our input variable
close_df= gg_df['close_price']

data_len = round(len(close_df) * 0.8)

close_df_train = close_df.loc[close_df.index < data_len]
close_df_test = close_df.loc[close_df.index >= data_len]

print(close_df_train.head())
print()
print(close_df_test.head())

In [ ]:
# Scale the training data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(close_df_train.values.reshape(-1,1))
scaled_data.shape

In [ ]:
# Use the previous 30 days 'close' price to predict the 31st day's price
previous_days = 30

In [ ]:
# Splite data into X_train and y_train
X_train = []
y_train = []

for i in range(previous_days, len(scaled_data)):
    X_train.append(scaled_data[i-previous_days:i, 0])
    y_train.append(scaled_data[i, 0])

In [ ]:
# Convert the X_train and y_train to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train.shape)
print(y_train.shape)

In [ ]:
# Reshape the X_train since LSTM needs 3-dimensional inputs
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

In [ ]:
# Build the LSTM model
model = Sequential()
# Add LSTM layer
model.add(LSTM(units = 50, return_sequences=True, input_shape = (X_train.shape[1],1)))
model.add(LSTM(units = 50, return_sequences=True))
model.add(LSTM(units = 50))
# prediction output layer
model.add(Dense(units=1))

In [ ]:
# compile and train the data
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size =32, epochs=20)

In [ ]:
model.summary()

In [ ]:
#### Test model ####

# Scale the testing data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_test_data = scaler.fit_transform(close_df_test.values.reshape(-1,1))
scaled_test_data.shape

In [ ]:
# Create X_test and y_test
X_test = []
y_test = []

for i in range(previous_days, len(scaled_test_data)):
    X_test.append(scaled_test_data[i-previous_days:i, 0])
    y_test.append(scaled_test_data[i, 0])

In [ ]:
# Convert the X_test to numpy arrays
X_test = np.array(X_test)
print(X_test.shape)

In [ ]:
# reshape
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print(X_test.shape)

In [ ]:
# Predict the data

prediction = model.predict(X_test)
prediction = scaler.inverse_transform(prediction)

In [ ]:
# plot test predictions

plt.figure(figsize=(12, 8))
plt.plot(y_test, color='red', label='Actual')
plt.plot(prediction, color='green', label='Predicted')
plt.xlabel('Date')
plt.ylabel('Stock Price')

plt.title("Stock Price predicted by LSTM Model")
plt.grid()
plt.legend()
plt.show()